In [1]:
# Import necessary libraries;
import numpy as np
from numpy import mean
from numpy import std


import pandas as pd

import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting defaults
import seaborn as sns; sns.set_style('white')

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


from mpl_toolkits.mplot3d import Axes3D
from numpy import genfromtxt

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.datasets import load_digits, make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler 
from sklearn.preprocessing import RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import seaborn as sns
from sklearn import tree

In [2]:
fireWall = pd.read_csv('log2.csv')

fireWallXData = fireWall[['Source Port', 'Destination Port','NAT Source Port',
                         'NAT Destination Port','Bytes','Bytes Sent','Bytes Received','Packets',
                        'Elapsed Time (sec)','pkts_sent','pkts_received']]
fireWallyData = fireWall[['Action']]

X = fireWallXData.iloc[:,0:10]
y = fireWallyData.iloc[:,0]


print(y.shape)
print(X.shape)

#print(fireWall.shape)

FireWALLData = pd.concat([X, y], axis=1)
print(FireWALLData.shape)

newX = FireWALLData.iloc[:,0:9]
newy = FireWALLData.iloc[:,10]

from sklearn.preprocessing import LabelEncoder
encodeLabel = LabelEncoder()
newy = encodeLabel.fit_transform(newy)
print(newy)


FireWALLData.loc[FireWALLData['Action'] == 0, 'Action'] = 1
FireWALLData.loc[FireWALLData['Action'] == 1, 'Action'] = 0
FireWALLData.loc[FireWALLData['Action'] == 3, 'Action'] = 0
FireWALLData.loc[FireWALLData['Action'] == 4, 'Action'] = 0

FireWALLData.head()



(65532,)
(65532, 10)
(65532, 11)
[0 0 0 ... 2 2 2]


,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,Action
0,57222,53,54587,53,177,94,83,2,30,1,allow
1,56258,3389,56258,3389,4768,1600,3168,19,17,10,allow
2,6881,50321,43265,50321,238,118,120,2,1199,1,allow
3,50553,3389,50553,3389,3327,1438,1889,15,17,8,allow
4,50002,443,45848,443,25358,6778,18580,31,16,13,allow


In [3]:
X_train, X_test, y_train, y_test = train_test_split( newX, newy, test_size=60000, random_state=1738)



# MULTI LAYERED PERCEPTRON TRIAL ONE ON FIRE WALL DATASET

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', MLPClassifier(max_iter = 250))])

    search_space = [
                {
                 
                 'classifier': [MLPClassifier(max_iter = 250)],
                 'classifier__hidden_layer_sizes': [(5,),(10,),(15,),(20,)],
                 'classifier__activation': ['logistic','tanh'],
                 'classifier__solver': ['lbfgs'],
                 'classifier__alpha': [0.0001,0.001,0.01,.1],
                 'classifier__learning_rate': ['constant','invscaling','adaptive']},
                
                {
                 
                  
                 'classifier': [MLPClassifier(max_iter = 250)],
                 'classifier__hidden_layer_sizes': [(5,),(10,),(15,),(20,)],
                 'classifier__activation': ['logistic','tanh'],
                 'classifier__solver': ['sgd'],
                 'classifier__alpha': [0.0001,0.001,0.01,.1],
                 'classifier__learning_rate': ['constant','invscaling','adaptive']},
                
                
                {
                
                 
                 'classifier': [MLPClassifier(max_iter = 250)],
                 'classifier__hidden_layer_sizes': [(5,),(10,),(15,),(20,)],
                 'classifier__activation': ['logistic','tanh'],
                 'classifier__solver': ['adam'],
                 'classifier__alpha': [0.0001,0.001,0.01,.1],
                 'classifier__learning_rate': ['constant','invscaling','adaptive']},
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_1_MLP = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL ONE RESULTS ---------------------------------------------")

Fitting 2 folds for each of 288 candidates, totalling 576 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL ONE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 576 out of 576 | elapsed:  1.1min finished


In [6]:
TRIAL_1_MLP.cv_results_

{'mean_fit_time': array([0.4984293 , 0.71811748, 0.53846347, 0.71061039, 0.65581536,
        0.70735872, 0.84147406, 1.03163695, 1.01362216, 1.12872076,
        1.09969616, 1.0719223 , 0.68633974, 0.53245974, 0.68433917,
        0.85298252, 0.83947313, 0.83071518, 0.98459625, 0.98259461,
        1.07142174, 1.15349197, 1.08067918, 1.0351392 , 0.66982651,
        0.47816181, 0.67683256, 0.80344045, 0.80444431, 0.76440704,
        0.71636546, 0.92529583, 0.9140377 , 0.99360526, 0.9983598 ,
        1.02212989, 0.37807977, 0.30276036, 0.37582445, 0.53120685,
        0.52795649, 0.44212985, 0.67007673, 0.67382932, 0.54446828,
        0.60577238, 0.68158662, 0.68959379, 0.64505458, 0.61277759,
        0.74238801, 0.93605614, 0.88776374, 0.91478705, 1.11746156,
        1.1292218 , 1.15023971, 1.29786742, 1.23806453, 1.26859093,
        0.56823874, 0.75289583, 0.72988009, 0.89376807, 0.90502691,
        0.91203558, 1.10444951, 1.11495972, 1.14523423, 1.25532913,
        1.21854627, 1.25708163,

In [7]:
print("---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_1_MLP.cv_results_['rank_test_neg_log_loss']
TRIAL_1_MLP.cv_results_['params'][ np.argmin(TRIAL_1_MLP.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': MLPClassifier(max_iter=250),
 'classifier__activation': 'logistic',
 'classifier__alpha': 0.01,
 'classifier__hidden_layer_sizes': (5,),
 'classifier__learning_rate': 'constant',
 'classifier__solver': 'lbfgs'}

In [8]:
print("---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------")
TRIAL_1_MLP.cv_results_['rank_test_f1_micro']
TRIAL_1_MLP.cv_results_['params'][ np.argmin(TRIAL_1_MLP.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------


{'classifier': MLPClassifier(max_iter=250),
 'classifier__activation': 'logistic',
 'classifier__alpha': 0.001,
 'classifier__hidden_layer_sizes': (5,),
 'classifier__learning_rate': 'constant',
 'classifier__solver': 'lbfgs'}

In [9]:
print("---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------")
TRIAL_1_MLP.cv_results_['rank_test_roc_auc_ovo']
TRIAL_1_MLP.cv_results_['params'][ np.argmin(TRIAL_1_MLP.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------


{'classifier': MLPClassifier(max_iter=250),
 'classifier__activation': 'logistic',
 'classifier__alpha': 0.001,
 'classifier__hidden_layer_sizes': (20,),
 'classifier__learning_rate': 'constant',
 'classifier__solver': 'lbfgs'}

In [10]:
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_1_MLP.cv_results_['params'][ np.argmin(TRIAL_1_MLP.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 1 MLP using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_1_MLP.cv_results_['params'][ np.argmin(TRIAL_1_MLP.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 1 MLP using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_1_MLP.cv_results_['params'][ np.argmin(TRIAL_1_MLP.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 1 MLP using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 1 MLP using best NEG LOG LOSS hyperparameters :0.9923
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 MLP using best F1 hyperparameters :0.9958166666666667
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 MLP using best ROC_AUC hyperparameters :0.99545
------------------------------------------------------------------------------------------------


# MULTI LAYERED PERCEPTRON TRIAL TWO ON FIRE WALLDATASET

In [11]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    pipe = Pipeline([('std', StandardScaler()),('classifier', MLPClassifier(max_iter = 250))])

    search_space = [
                {
                 
                 'classifier': [MLPClassifier(max_iter = 250)],
                 'classifier__hidden_layer_sizes': [(5,),(10,),(15,),(20,)],
                 'classifier__activation': ['logistic','tanh'],
                 'classifier__solver': ['lbfgs'],
                 'classifier__alpha': [0.0001,0.001,0.01,.1],
                 'classifier__learning_rate': ['constant','invscaling','adaptive']},
                
                {
                 
                  
                 'classifier': [MLPClassifier(max_iter = 250)],
                 'classifier__hidden_layer_sizes': [(5,),(10,),(15,),(20,)],
                 'classifier__activation': ['logistic','tanh'],
                 'classifier__solver': ['sgd'],
                 'classifier__alpha': [0.0001,0.001,0.01,.1],
                 'classifier__learning_rate': ['constant','invscaling','adaptive']},
                
                
                {
                
                 
                 'classifier': [MLPClassifier(max_iter = 250)],
                 'classifier__hidden_layer_sizes': [(5,),(10,),(15,),(20,)],
                 'classifier__activation': ['logistic','tanh'],
                 'classifier__solver': ['adam'],
                 'classifier__alpha': [0.0001,0.001,0.01,.1],
                 'classifier__learning_rate': ['constant','invscaling','adaptive']},
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_2_MLP = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL TWO RESULTS ---------------------------------------------")

Fitting 2 folds for each of 288 candidates, totalling 576 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL TWO RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 576 out of 576 | elapsed:  1.1min finished


In [12]:
TRIAL_2_MLP.cv_results_

{'mean_fit_time': array([0.29675651, 0.48391616, 0.63629746, 0.74939549, 0.51994848,
        0.68709195, 0.83797133, 0.98284626, 0.7956841 , 0.9375571 ,
        1.01437306, 0.97083473, 0.37307   , 0.50668895, 0.54046547,
        0.71061182, 0.79743564, 0.60026622, 0.80043852, 0.77491665,
        0.97634041, 1.09118807, 0.89276862, 1.05615783, 0.49167085,
        0.37457275, 0.52695382, 0.5677402 , 0.65706563, 0.62678957,
        0.69184661, 0.84022248, 0.73488224, 0.75014544, 0.84697843,
        0.82596028, 0.52770483, 0.40709889, 0.37081909, 0.62754083,
        0.58050156, 0.51369226, 0.57299316, 0.56323552, 0.60351908,
        0.71611726, 0.6235348 , 0.64980912, 0.54521656, 0.75314665,
        0.55272424, 0.57299304, 0.87099957, 0.95157027, 1.1134578 ,
        1.08143151, 0.88225937, 1.05115187, 1.17125595, 1.00986743,
        0.54296482, 0.73613381, 0.59476268, 0.91303635, 0.69109356,
        0.69810081, 1.0201267 , 1.14498472, 1.18802214, 1.33164489,
        1.3341465 , 1.38143826,

In [13]:
print("---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_2_MLP.cv_results_['rank_test_neg_log_loss']
TRIAL_2_MLP.cv_results_['params'][ np.argmin(TRIAL_2_MLP.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': MLPClassifier(max_iter=250),
 'classifier__activation': 'tanh',
 'classifier__alpha': 0.001,
 'classifier__hidden_layer_sizes': (5,),
 'classifier__learning_rate': 'adaptive',
 'classifier__solver': 'lbfgs'}

In [14]:
print("---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------")
TRIAL_2_MLP.cv_results_['rank_test_f1_micro']
TRIAL_2_MLP.cv_results_['params'][ np.argmin(TRIAL_2_MLP.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------


{'classifier': MLPClassifier(max_iter=250),
 'classifier__activation': 'tanh',
 'classifier__alpha': 0.001,
 'classifier__hidden_layer_sizes': (5,),
 'classifier__learning_rate': 'adaptive',
 'classifier__solver': 'lbfgs'}

In [15]:
print("---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------")
TRIAL_2_MLP.cv_results_['rank_test_roc_auc_ovo']
TRIAL_2_MLP.cv_results_['params'][ np.argmin(TRIAL_2_MLP.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------


{'classifier': MLPClassifier(max_iter=250),
 'classifier__activation': 'logistic',
 'classifier__alpha': 0.001,
 'classifier__hidden_layer_sizes': (15,),
 'classifier__learning_rate': 'constant',
 'classifier__solver': 'lbfgs'}

In [16]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_2_MLP.cv_results_['params'][ np.argmin(TRIAL_2_MLP.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 2 MLP using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_2_MLP.cv_results_['params'][ np.argmin(TRIAL_2_MLP.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 2 MLP using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_2_MLP.cv_results_['params'][ np.argmin(TRIAL_2_MLP.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 2 MLP using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 2 MLP using best NEG LOG LOSS hyperparameters :0.9928666666666667
------------------------------------------------------------------------------------------------
Accuracy of Trial 2 MLP using best F1 hyperparameters :0.9962
------------------------------------------------------------------------------------------------
Accuracy of Trial 2 MLP using best ROC_AUC hyperparameters :0.9965
------------------------------------------------------------------------------------------------


# MULTI LAYERED PERCEPTRON TRIAL THREE ON FIRE WALL DATASET

In [17]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    # Run GridSearch to find best hyperparameters per 5000 data points 
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', MLPClassifier(max_iter = 250))])

    search_space = [
                {
                 
                 'classifier': [MLPClassifier(max_iter = 250)],
                 'classifier__hidden_layer_sizes': [(5,),(10,),(15,),(20,)],
                 'classifier__activation': ['logistic','tanh'],
                 'classifier__solver': ['lbfgs'],
                 'classifier__alpha': [0.0001,0.001,0.01,.1],
                 'classifier__learning_rate': ['constant','invscaling','adaptive']},
                
                {
                 
                  
                 'classifier': [MLPClassifier(max_iter = 250)],
                 'classifier__hidden_layer_sizes': [(5,),(10,),(15,),(20,)],
                 'classifier__activation': ['logistic','tanh'],
                 'classifier__solver': ['sgd'],
                 'classifier__alpha': [0.0001,0.001,0.01,.1],
                 'classifier__learning_rate': ['constant','invscaling','adaptive']},
                
                
                {
                
                 
                 'classifier': [MLPClassifier(max_iter = 250)],
                 'classifier__hidden_layer_sizes': [(5,),(10,),(15,),(20,)],
                 'classifier__activation': ['logistic','tanh'],
                 'classifier__solver': ['adam'],
                 'classifier__alpha': [0.0001,0.001,0.01,.1],
                 'classifier__learning_rate': ['constant','invscaling','adaptive']},
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_3_MLP = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL THREE RESULTS ---------------------------------------------")

Fitting 2 folds for each of 288 candidates, totalling 576 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   15.4s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL THREE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 576 out of 576 | elapsed:  1.2min finished


In [18]:
TRIAL_3_MLP.cv_results_

{'mean_fit_time': array([0.36231291, 0.52620339, 0.5124402 , 0.78767884, 0.49817944,
        0.78893197, 0.91203606, 0.97333837, 1.02513182, 1.11445808,
        1.09394145, 1.07117164, 0.71261287, 0.63279343, 0.66507304,
        0.89527202, 0.67132819, 0.94256079, 0.98885071, 1.06241417,
        1.08943701, 1.22580612, 1.19102275, 1.23431242, 0.79643512,
        0.65856683, 0.72412169, 1.00085998, 0.89226758, 0.8604908 ,
        1.0541563 , 1.09419012, 1.05265462, 1.27134395, 1.15449309,
        1.01762474, 0.46339929, 0.60577142, 0.66131818, 0.53621197,
        0.58775651, 0.68508899, 0.97158563, 0.81670177, 0.93180227,
        0.89251828, 0.76040363, 0.91978896, 0.87200034, 0.85423398,
        0.67908418, 1.00336242, 0.98834991, 1.0138731 , 1.24882412,
        1.22080028, 1.17926431, 1.41446686, 1.28910732, 1.31312883,
        0.75064898, 0.70560622, 0.72412276, 1.00060976, 1.05265594,
        0.9518187 , 1.33865106, 1.16675436, 1.24982727, 1.44549394,
        1.50254261, 1.47977173,

In [19]:
print("---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_3_MLP.cv_results_['rank_test_neg_log_loss']
TRIAL_3_MLP.cv_results_['params'][ np.argmin(TRIAL_3_MLP.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': MLPClassifier(max_iter=250),
 'classifier__activation': 'logistic',
 'classifier__alpha': 0.01,
 'classifier__hidden_layer_sizes': (5,),
 'classifier__learning_rate': 'constant',
 'classifier__solver': 'lbfgs'}

In [20]:
print("---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------")
TRIAL_3_MLP.cv_results_['rank_test_f1_micro']
TRIAL_3_MLP.cv_results_['params'][ np.argmin(TRIAL_3_MLP.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------


{'classifier': MLPClassifier(max_iter=250),
 'classifier__activation': 'tanh',
 'classifier__alpha': 0.001,
 'classifier__hidden_layer_sizes': (5,),
 'classifier__learning_rate': 'constant',
 'classifier__solver': 'lbfgs'}

In [21]:
print("---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------")
TRIAL_3_MLP.cv_results_['rank_test_roc_auc_ovo']
TRIAL_3_MLP.cv_results_['params'][ np.argmin(TRIAL_3_MLP.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------


{'classifier': MLPClassifier(max_iter=250),
 'classifier__activation': 'tanh',
 'classifier__alpha': 0.001,
 'classifier__hidden_layer_sizes': (20,),
 'classifier__learning_rate': 'invscaling',
 'classifier__solver': 'lbfgs'}

In [22]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_3_MLP.cv_results_['params'][ np.argmin(TRIAL_3_MLP.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 3 MLP using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_3_MLP.cv_results_['params'][ np.argmin(TRIAL_3_MLP.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 3 MLP using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_3_MLP.cv_results_['params'][ np.argmin(TRIAL_3_MLP.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 3 MLP using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 3 MLP using best NEG LOG LOSS hyperparameters :0.9965
------------------------------------------------------------------------------------------------
Accuracy of Trial 3 MLP using best F1 hyperparameters :0.9964833333333334
------------------------------------------------------------------------------------------------
Accuracy of Trial 3 MLP using best ROC_AUC hyperparameters :0.99625
------------------------------------------------------------------------------------------------


# MULTI LAYERED PERCEPTRON TRIAL FOUR ON FIRE WALL DATASET

In [23]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', MLPClassifier(max_iter = 250))])

    search_space = [
                {
                 
                 'classifier': [MLPClassifier(max_iter = 250)],
                 'classifier__hidden_layer_sizes': [(5,),(10,),(15,),(20,)],
                 'classifier__activation': ['logistic','tanh'],
                 'classifier__solver': ['lbfgs'],
                 'classifier__alpha': [0.0001,0.001,0.01,.1],
                 'classifier__learning_rate': ['constant','invscaling','adaptive']},
                
                {
                 
                  
                 'classifier': [MLPClassifier(max_iter = 250)],
                 'classifier__hidden_layer_sizes': [(5,),(10,),(15,),(20,)],
                 'classifier__activation': ['logistic','tanh'],
                 'classifier__solver': ['sgd'],
                 'classifier__alpha': [0.0001,0.001,0.01,.1],
                 'classifier__learning_rate': ['constant','invscaling','adaptive']},
                
                
                {
                
                 
                 'classifier': [MLPClassifier(max_iter = 250)],
                 'classifier__hidden_layer_sizes': [(5,),(10,),(15,),(20,)],
                 'classifier__activation': ['logistic','tanh'],
                 'classifier__solver': ['adam'],
                 'classifier__alpha': [0.0001,0.001,0.01,.1],
                 'classifier__learning_rate': ['constant','invscaling','adaptive']},
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_4_MLP = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FOUR RESULTS ---------------------------------------------")

Fitting 2 folds for each of 288 candidates, totalling 576 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL FOUR RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 576 out of 576 | elapsed:  1.2min finished


In [24]:
TRIAL_4_MLP.cv_results_

{'mean_fit_time': array([0.87699342, 0.4613986 , 0.54346836, 0.68058562, 1.0263828 ,
        1.03313887, 0.89001679, 0.81720245, 0.87925756, 1.33689845,
        1.29086065, 1.42472565, 0.98209453, 0.63329422, 0.57099056,
        1.02463102, 1.0151248 , 0.95382297, 0.79293191, 1.34615612,
        1.13622642, 1.30962622, 1.31563139, 1.45625341, 0.75739992,
        0.83621871, 0.73037803, 0.97133613, 0.88275945, 0.9825958 ,
        1.10119665, 1.14348304, 1.14648545, 1.20553601, 1.23255885,
        1.01262116, 0.52169979, 0.54722095, 0.57049   , 0.78817928,
        0.76891136, 0.85034847, 0.88401163, 0.92629921, 0.90703022,
        0.91303444, 0.80769408, 0.78767884, 0.8920182 , 0.85348368,
        0.81194901, 1.25482857, 1.10294998, 1.00436497, 1.34498727,
        1.39619994, 1.37968588, 1.5538367 , 1.52931488, 1.52806532,
        0.84372556, 0.88651156, 0.93680573, 1.06916928, 1.07867646,
        1.16625535, 1.53431976, 1.35341311, 1.34890974, 1.49603677,
        1.4439919 , 1.52356207,

In [25]:
print("---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_4_MLP.cv_results_['rank_test_neg_log_loss']
TRIAL_4_MLP.cv_results_['params'][ np.argmin(TRIAL_4_MLP.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': MLPClassifier(max_iter=250),
 'classifier__activation': 'tanh',
 'classifier__alpha': 0.001,
 'classifier__hidden_layer_sizes': (10,),
 'classifier__learning_rate': 'adaptive',
 'classifier__solver': 'lbfgs'}

In [26]:
print("---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------")
TRIAL_4_MLP.cv_results_['rank_test_f1_micro']
TRIAL_4_MLP.cv_results_['params'][ np.argmin(TRIAL_4_MLP.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------


{'classifier': MLPClassifier(max_iter=250),
 'classifier__activation': 'logistic',
 'classifier__alpha': 0.0001,
 'classifier__hidden_layer_sizes': (5,),
 'classifier__learning_rate': 'constant',
 'classifier__solver': 'lbfgs'}

In [27]:
print("---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------")
TRIAL_4_MLP.cv_results_['rank_test_roc_auc_ovo']
TRIAL_4_MLP.cv_results_['params'][ np.argmin(TRIAL_4_MLP.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------


{'classifier': MLPClassifier(max_iter=250),
 'classifier__activation': 'logistic',
 'classifier__alpha': 0.001,
 'classifier__hidden_layer_sizes': (10,),
 'classifier__learning_rate': 'invscaling',
 'classifier__solver': 'lbfgs'}

In [28]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_4_MLP.cv_results_['params'][ np.argmin(TRIAL_4_MLP.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 4 MPL using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_4_MLP.cv_results_['params'][ np.argmin(TRIAL_4_MLP.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 4 MPL using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_4_MLP.cv_results_['params'][ np.argmin(TRIAL_4_MLP.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 4 MPL using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 4 MPL using best NEG LOG LOSS hyperparameters :0.99265
------------------------------------------------------------------------------------------------
Accuracy of Trial 4 MPL using best F1 hyperparameters :0.9958833333333333
------------------------------------------------------------------------------------------------
Accuracy of Trial 4 MPL using best ROC_AUC hyperparameters :0.9958333333333333
------------------------------------------------------------------------------------------------


# MULTI LAYERED PERCEPTRON TRIAL FIVE ON FIRE WALL DATASET

In [29]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    # Run GridSearch to find best hyperparameters per 5000 data points 
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', MLPClassifier(max_iter = 250))])

    search_space = [
                {
                 
                 'classifier': [MLPClassifier(max_iter = 250)],
                 'classifier__hidden_layer_sizes': [(5,),(10,),(15,),(20,)],
                 'classifier__activation': ['logistic','tanh'],
                 'classifier__solver': ['lbfgs'],
                 'classifier__alpha': [0.0001,0.001,0.01,.1],
                 'classifier__learning_rate': ['constant','invscaling','adaptive']},
                
                {
                 
                  
                 'classifier': [MLPClassifier(max_iter = 250)],
                 'classifier__hidden_layer_sizes': [(5,),(10,),(15,),(20,)],
                 'classifier__activation': ['logistic','tanh'],
                 'classifier__solver': ['sgd'],
                 'classifier__alpha': [0.0001,0.001,0.01,.1],
                 'classifier__learning_rate': ['constant','invscaling','adaptive']},
                
                
                {
                
                 
                 'classifier': [MLPClassifier(max_iter = 250)],
                 'classifier__hidden_layer_sizes': [(5,),(10,),(15,),(20,)],
                 'classifier__activation': ['logistic','tanh'],
                 'classifier__solver': ['adam'],
                 'classifier__alpha': [0.0001,0.001,0.01,.1],
                 'classifier__learning_rate': ['constant','invscaling','adaptive']},
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_5_MLP = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FIVE RESULTS ---------------------------------------------")

Fitting 2 folds for each of 288 candidates, totalling 576 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   14.9s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL FIVE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 576 out of 576 | elapsed:  1.1min finished


In [30]:
TRIAL_5_MLP.cv_results_

{'mean_fit_time': array([0.41335738, 0.67107761, 0.63029349, 0.66182053, 0.91453743,
        0.88301027, 1.14248419, 1.02638459, 0.95281911, 0.91728866,
        1.1777637 , 1.09769452, 0.73112917, 0.74363947, 0.78917682,
        0.88726354, 0.76791096, 0.90627956, 1.00336158, 1.08368254,
        0.57449436, 1.11696017, 1.10344875, 1.14473438, 0.72962832,
        0.61678004, 0.68859267, 0.8389715 , 0.83596849, 0.8111974 ,
        0.9605757 , 1.02738571, 1.02363038, 1.08893645, 1.09494233,
        1.08618367, 0.55697799, 0.61627924, 0.47515798, 0.67783237,
        0.80844545, 0.8129487 , 0.89276898, 0.93905818, 0.85173333,
        0.7446413 , 0.86199129, 0.9300499 , 0.52895534, 0.53420961,
        0.67082787, 0.97708964, 0.99885941, 0.88801336, 1.21229196,
        1.13097262, 1.13722801, 1.3248893 , 1.33539724, 1.30462289,
        0.78342533, 0.78442562, 0.65981805, 1.03188848, 0.80644608,
        0.95056856, 1.1274699 , 1.17976618, 1.14848781, 1.30437326,
        1.30962718, 1.31312859,

In [31]:
print("---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_5_MLP.cv_results_['rank_test_neg_log_loss']
TRIAL_5_MLP.cv_results_['params'][ np.argmin(TRIAL_5_MLP.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': MLPClassifier(max_iter=250),
 'classifier__activation': 'logistic',
 'classifier__alpha': 0.01,
 'classifier__hidden_layer_sizes': (10,),
 'classifier__learning_rate': 'adaptive',
 'classifier__solver': 'lbfgs'}

In [32]:
print("---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------")
TRIAL_5_MLP.cv_results_['rank_test_f1_micro']
TRIAL_5_MLP.cv_results_['params'][ np.argmin(TRIAL_5_MLP.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------


{'classifier': MLPClassifier(max_iter=250),
 'classifier__activation': 'logistic',
 'classifier__alpha': 0.001,
 'classifier__hidden_layer_sizes': (20,),
 'classifier__learning_rate': 'constant',
 'classifier__solver': 'lbfgs'}

In [33]:
print("---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------")
TRIAL_5_MLP.cv_results_['rank_test_roc_auc_ovo']
TRIAL_5_MLP.cv_results_['params'][ np.argmin(TRIAL_5_MLP.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------


{'classifier': MLPClassifier(max_iter=250),
 'classifier__activation': 'tanh',
 'classifier__alpha': 0.001,
 'classifier__hidden_layer_sizes': (10,),
 'classifier__learning_rate': 'invscaling',
 'classifier__solver': 'lbfgs'}

In [34]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_5_MLP.cv_results_['params'][ np.argmin(TRIAL_5_MLP.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 5 MLP using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_5_MLP.cv_results_['params'][ np.argmin(TRIAL_5_MLP.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 5 MLP using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_5_MLP.cv_results_['params'][ np.argmin(TRIAL_5_MLP.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 5 MLP using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 5 MLP using best NEG LOG LOSS hyperparameters :0.9952666666666666
------------------------------------------------------------------------------------------------
Accuracy of Trial 5 MLP using best F1 hyperparameters :0.9967166666666667
------------------------------------------------------------------------------------------------
Accuracy of Trial 5 MLP using best ROC_AUC hyperparameters :0.9966166666666667
------------------------------------------------------------------------------------------------
